In [2]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
import json, sys, os
from pymatgen.core.lattice import Lattice
from pymatgen.core.structure import Structure
from pymatgen.core import Composition
import matplotlib.pyplot as plt
import mlcpquicker
import mlcpquick
from tqdm import tqdm
import importlib
import plotly.express as px
import seaborn as sns

import helperfunctions as hf
import reference as ref
import reference2 as ref2

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, HistGradientBoostingClassifier,HistGradientBoostingRegressor,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,mean_squared_error,mean_absolute_error,ConfusionMatrixDisplay, r2_score, f1_score,roc_curve, roc_auc_score


In [139]:
importlib.reload(mlcpquicker)
importlib.reload(mlcpquick)
importlib.reload(ref)
importlib.reload(hf)
importlib.reload(ref2)

<module 'reference2' from '/Users/aboth/Desktop/gordon/mlcpproject/mlcp code/reference2.py'>

In [140]:
#pandas view settings
pd.set_option('display.max_rows',60)
pd.set_option('display.min_rows',50)
pd.set_option('display.max_columns',None)

In [141]:
#url construction
SERVERAPI="http://aflowlib.duke.edu/search/API/?"
#without conditions, just a comma separated list
SUMMONS="agl_thermal_conductivity_300K(0*),agl_gruneisen,agl_debye,entropic_temperature,natoms,nspecies,enthalpy_formation_atom,ael_speed_sound_average,energy_atom,ael_bulk_modulus_vrh,positions_fractional,aflow_prototype_label_relax,geometry,geometry_orig,species,Pearson_symbol_relax,spacegroup_relax,spacegroup_orig,Wyckoff_letters,Wyckoff_letters_orig,Wyckoff_multiplicities,Wyckoff_multiplicities_orig,Wyckoff_site_symmetries,Egap,Egap_fit,Egap_type,aurl,catalog('icsd'),$paging(0)"
response=json.loads(urlopen(SERVERAPI+SUMMONS).read().decode( "utf-8" ))
#a list for each property/data recieved
compounds=[]
auids=[]
aurls=[]
kappa2=[]
positions2=[]
debye=[]
gruneisen=[]
enthalpy_atom=[]
bulk_modulus=[]
energy=[]
speed_sound=[]
prototype=[]
Wletters2=[]
Wmultiplicities2=[]
Wsite_symmetries2=[]
#from 'geometry'
spacegroup=[]
Pearson=[]
species2=[]
geometry2=[]
#original geometry
real_geometry2=[]
real_spacegroup=[]
real_Wletters2=[]
real_Wmultiplicities2=[]
nspecies2=[]
temperature2=[]
Egap2=[]
Egap_fit2=[]
Egap_type=[]
natoms2=[]

#putting the stuff in
for datum in response:
    compounds.append(datum['compound'])
    auids.append(datum['auid'])
    positions2.append(datum['positions_fractional'])
    debye.append(datum['agl_debye'])
    gruneisen.append(datum['agl_gruneisen'])
    enthalpy_atom.append(datum['enthalpy_formation_atom'])
    bulk_modulus.append(datum['ael_bulk_modulus_vrh'])
    energy.append(datum['energy_atom'])
    speed_sound.append(datum['ael_speed_sound_average'])    
    kappa2.append(datum['agl_thermal_conductivity_300K'])
    prototype.append(datum['aflow_prototype_label_relax'])
    Wletters2.append(datum['Wyckoff_letters'])
    Wmultiplicities2.append(datum['Wyckoff_multiplicities'])
    Wsite_symmetries2.append(datum['Wyckoff_site_symmetries'])
    spacegroup.append(datum['spacegroup_relax'])
    Pearson.append(datum['Pearson_symbol_relax'])
    species2.append(datum['species'])
    nspecies2.append(datum['nspecies'])
    geometry2.append(datum['geometry'])
    aurls.append(datum['aurl'])
    real_geometry2.append(datum['geometry_orig'])
    real_spacegroup.append(datum['spacegroup_orig'])
    real_Wletters2.append(datum['Wyckoff_letters_orig'])
    real_Wmultiplicities2.append(datum['Wyckoff_multiplicities_orig'])
    temperature2.append(datum['entropic_temperature'])
    Egap2.append(datum['Egap'])
    Egap_fit2.append(datum['Egap_fit'])
    Egap_type.append(datum['Egap_type'])
    natoms2.append(datum['natoms'])

In [142]:
#preprocessing
positions=[]
for i in positions2:
    pel=i.split(';')
    peach=[]
    for j in pel:
        pnum=[]
        for k in j.split(','):
            pnum.append(float(k))
        peach.append(pnum)
    positions.append(peach)
species=[]
for i in species2:
    species.append(i.split(','))
real_geometry=[]
for i in real_geometry2:
    set=[]
    for j in i.split(','):
        set.append(float(j))
    real_geometry.append(set)
geometry=[]
for i in geometry2:
    set=[]
    for j in i.split(','):
        set.append(float(j))
    geometry.append(set)
Wletters=[]
for i in Wletters2:
    pel=i.split(';')
    peach=[]
    for j in pel:
        peach.append(j.split(','))
    Wletters.append(peach)
real_Wletters=[]
for i in real_Wletters2:
    pel=i.split(';')
    peach=[]
    for j in pel:
        peach.append(j.split(','))
    real_Wletters.append(peach)
Wmultiplicities=[]
for i in Wmultiplicities2:
    pel=i.split(';')
    peach=[]
    for j in pel:
        peach.append(j.split(','))
    Wmultiplicities.append(peach)
real_Wmultiplicities=[]
for i in real_Wmultiplicities2:
    pel=i.split(';')
    peach=[]
    for j in pel:
        peach.append(j.split(','))
    real_Wmultiplicities.append(peach)
Wsite_symmetries=[]
for i in Wsite_symmetries2:
    pel=i.split(';')
    peach=[]
    for j in pel:
        peach.append(j.split(','))
    Wsite_symmetries.append(peach)
summult=[]
for i in Wmultiplicities:
    numbsum=0
    for j in i:
        for k in j:
            numbsum+=int(k)
    summult.append(numbsum)
real_summult=[]
for i in real_Wmultiplicities:
    numbsum=0
    for j in i:
        for k in j:
            numbsum+=int(k)
    real_summult.append(numbsum)
ICSD=[]
for i in range(len(aurls)):
    ICSD.append(aurls[i].split('ICSD_')[2])
nspecies=[]
for i in range(len(nspecies2)):
    nspecies.append(int(nspecies2[i]))
temperature=[]
for i in range(len(temperature2)):
    try:
        temperature.append(float(temperature2[i]))
    except:
        temperature.append(-1)
kappa=[]
for i in range(len(kappa2)):
    kappa.append(float(kappa2[i]))
Egap=[]
for i in range(len(Egap2)):
    Egap.append(float(Egap2[i]))
Egap_fit=[]
for i in range(len(Egap_fit2)):
    Egap_fit.append(float(Egap_fit2[i]))
Etype=[]
for i in Egap:
    if i==0:
        Evalue=0
    elif i<0.5:
        Evalue=1
    else:
        Evalue=2
    Etype.append(Evalue)
natoms=[]
for i in range(len(natoms2)):
    natoms.append(int(natoms2[i]))

In [143]:
AFLOWtherm = pd.DataFrame()

In [144]:
#adding to the dataframe
AFLOWtherm['compound']=compounds #numbers add to comp
AFLOWtherm['real_spacegroup']=real_spacegroup
AFLOWtherm['compoundtype']=AFLOWtherm['compound']+'_'+AFLOWtherm['real_spacegroup']
AFLOWtherm['ICSD']=ICSD
AFLOWtherm['kappa']=kappa
AFLOWtherm['bulk_modulus']=bulk_modulus
AFLOWtherm['debye']=debye
AFLOWtherm['gruneisen']=gruneisen
AFLOWtherm['enthalpy']=enthalpy_atom
AFLOWtherm['energy']=energy
AFLOWtherm['speed_sound']=speed_sound
AFLOWtherm['positions']=positions #comp
AFLOWtherm['prototype']=prototype
AFLOWtherm['spacegroup']=spacegroup
AFLOWtherm['Pearson']=Pearson
AFLOWtherm['geometry']=geometry #6
AFLOWtherm['real_geometry']=real_geometry #6
AFLOWtherm['species']=species #elem
AFLOWtherm['nspecies']=nspecies
AFLOWtherm['natoms']=natoms
AFLOWtherm['Wletters']=Wletters #elem
AFLOWtherm['real_Wletters']=real_Wletters #elem?
AFLOWtherm['Wmultiplicities']=Wmultiplicities #elem
AFLOWtherm['real_Wmultiplicities']=real_Wmultiplicities #elem?
AFLOWtherm['Wsite_symmetries']=Wsite_symmetries #elem
AFLOWtherm['lenpos']=[len(a) for a in positions] 
AFLOWtherm['summult']= summult
AFLOWtherm['real_summult']= real_summult
AFLOWtherm['real_multiplier']= AFLOWtherm['real_summult']/AFLOWtherm['lenpos']
AFLOWtherm['multiplier']= AFLOWtherm['summult']/AFLOWtherm['lenpos']
AFLOWtherm['temperature']=temperature
AFLOWtherm['Egap']=Egap
AFLOWtherm['Egap_fit']=Egap_fit
AFLOWtherm['Egap_type']=Egap_type
AFLOWtherm['Etype']=Etype
AFLOWtherm['auid']=auids
AFLOWtherm['aurl']=aurls
sites=[]
for i in range(len(AFLOWtherm['Wmultiplicities'])):
    atomlist=[]
    for y in range(len(AFLOWtherm['Wmultiplicities'][i])):
        for j in AFLOWtherm['Wmultiplicities'][i][y]:
            for k in range(int(j)):
                atomlist.append(AFLOWtherm['species'][i][y])
    sites.append(atomlist)
real_sites=[]
for i in range(len(AFLOWtherm['real_Wmultiplicities'])):
    atomlist=[]
    for y in range(len(AFLOWtherm['real_Wmultiplicities'][i])):
        for j in AFLOWtherm['real_Wmultiplicities'][i][y]:
            for k in range(int(j)):
                atomlist.append(AFLOWtherm['species'][i][y])
    real_sites.append(atomlist)
AFLOWtherm['sites']= sites
AFLOWtherm['real_sites']= real_sites

In [145]:
fixedsites=[]
sitecount=[]
for i in range(len(AFLOWtherm)):
    if AFLOWtherm['multiplier'].iloc[i]==1:
        fixedsites.append(AFLOWtherm['sites'].iloc[i])
        sitecount.append(len(AFLOWtherm['sites'].iloc[i]))
    elif AFLOWtherm['multiplier'].iloc[i]==2:
        fixedsites.append(AFLOWtherm['sites'].iloc[i][::2])
        sitecount.append(len(AFLOWtherm['sites'].iloc[i][::2]))
    elif AFLOWtherm['multiplier'].iloc[i]==4:
        fixedsites.append(AFLOWtherm['sites'].iloc[i][::4])
        sitecount.append(len(AFLOWtherm['sites'].iloc[i][::4]))
    else:
        fixedsites.append(AFLOWtherm['sites'].iloc[i])
        sitecount.append(len(AFLOWtherm['sites'].iloc[i]))
AFLOWtherm['fixedsites']=fixedsites
AFLOWtherm['sitecount']=sitecount
AFLOWtherm['fixedmultiplier']= AFLOWtherm['sitecount']/AFLOWtherm['lenpos']

In [146]:
fixedrealsites=[]
realsitecount=[]
for i in range(len(AFLOWtherm)):
    if AFLOWtherm['real_multiplier'].iloc[i]==1:
        fixedrealsites.append(AFLOWtherm['real_sites'].iloc[i])
        realsitecount.append(len(AFLOWtherm['real_sites'].iloc[i]))
    elif AFLOWtherm['real_multiplier'].iloc[i]==2:
        fixedrealsites.append(AFLOWtherm['real_sites'].iloc[i][::2])
        realsitecount.append(len(AFLOWtherm['real_sites'].iloc[i][::2]))
    elif AFLOWtherm['real_multiplier'].iloc[i]==4:
        fixedrealsites.append(AFLOWtherm['real_sites'].iloc[i][::4])
        realsitecount.append(len(AFLOWtherm['real_sites'].iloc[i][::4]))
    else:
        fixedrealsites.append(AFLOWtherm['real_sites'].iloc[i])
        realsitecount.append(len(AFLOWtherm['real_sites'].iloc[i]))
AFLOWtherm['fixedrealsites']=fixedrealsites
AFLOWtherm['realsitecount']=realsitecount
AFLOWtherm['fixedrealmultiplier']= AFLOWtherm['realsitecount']/AFLOWtherm['lenpos']

In [147]:
geometries={}
for i in range(len(AFLOWtherm)):
    geometries[AFLOWtherm['ICSD'][i]]=Lattice.from_dict({'a':AFLOWtherm['real_geometry'][i][0],
                                           'b':AFLOWtherm['real_geometry'][i][1],
                                           'c':AFLOWtherm['real_geometry'][i][2],
                                           'alpha':AFLOWtherm['real_geometry'][i][3],
                                           'beta':AFLOWtherm['real_geometry'][i][4],
                                           'gamma':AFLOWtherm['real_geometry'][i][5]})

In [148]:
geometriesrelaxed={}
for i in range(len(AFLOWtherm)):
    geometriesrelaxed[AFLOWtherm['ICSD'][i]]=Lattice.from_dict({'a':AFLOWtherm['geometry'][i][0],
                                           'b':AFLOWtherm['geometry'][i][1],
                                           'c':AFLOWtherm['geometry'][i][2],
                                           'alpha':AFLOWtherm['geometry'][i][3],
                                           'beta':AFLOWtherm['geometry'][i][4],
                                           'gamma':AFLOWtherm['geometry'][i][5]})


In [50]:
# mlcpaflow={}
# mlcpproblems=[]
# for i in range(len(AFLOWtherm)):
#     try:
#         struct=Structure(geometries[AFLOWtherm['ICSD'][i]], AFLOWtherm['fixedrealsites'][i], AFLOWtherm['positions'][i])
#         mlcpaflow[AFLOWtherm['compound'][i]+'_'+AFLOWtherm['ICSD'][i]]=mlcpquicker.mlcp(struct,AFLOWtherm['compound'][i]+'_'+AFLOWtherm['ICSD'][i])
#         print(str(i)+AFLOWtherm['compound'][i]+'good')
#         mlcpproblems.append(0)
#     except:
#         mlcpproblems.append(1)
#         print(str(i)+AFLOWtherm['compound'][i]+'error')
#         mlcpaflow[AFLOWtherm['compound'][i]+'_'+AFLOWtherm['ICSD'][i]]=0

In [51]:
# for comp in mlcpaflow.keys():
#     if mlcpaflow[comp]==0:
#         continue
#     else:
#         Xs=[]
#         Xp=[]
#         Xd=[]
#         Xf=[]
#         Xg=[]
#         for atom in range(len(mlcpaflow[comp]['sites'])):
#             squaredcoeffs=[]
#             for con in mlcpaflow[comp]['coeffs'][atom]:
#                 squaredcoeffs.append(con**2)
#             sumcoeffs=sum(squaredcoeffs)
#             if sumcoeffs==0:
#                 Xs.append(0)
#                 Xp.append(0)
#                 Xd.append(0)
#                 Xf.append(0)
#                 Xg.append(0)
#             else:
#                 Xs.append(squaredcoeffs[0]/sumcoeffs)
#                 Xp.append(sum(squaredcoeffs[1:4])/sumcoeffs)
#                 Xd.append(sum(squaredcoeffs[4:9])/sumcoeffs)
#                 Xf.append(sum(squaredcoeffs[9:16])/sumcoeffs)
#                 Xg.append(sum(squaredcoeffs[16:25])/sumcoeffs)
#         mlcpaflow[comp]['Xs']=Xs
#         mlcpaflow[comp]['Xp']=Xp
#         mlcpaflow[comp]['Xd']=Xd
#         mlcpaflow[comp]['Xf']=Xf
#         mlcpaflow[comp]['Xg']=Xg

In [52]:
# np.save('mlcpdatafixed.npy', mlcpaflow)

In [53]:
# mlcpaflowrelax={}
# mlcpproblemsrelax=[]
# for i in range(len(AFLOWtherm)):
#     try:
#         struct=Structure(geometriesrelaxed[AFLOWtherm['ICSD'][i]], AFLOWtherm['fixedsites'][i], AFLOWtherm['positions'][i])
#         mlcpaflowrelax[AFLOWtherm['compound'][i]+'_'+AFLOWtherm['ICSD'][i]]=mlcpquicker.mlcp(struct,AFLOWtherm['compound'][i]+'_'+AFLOWtherm['ICSD'][i])
#         print(str(i)+AFLOWtherm['compound'][i]+'good')
#         mlcpproblemsrelax.append(0)
#     except:
#         mlcpproblemsrelax.append(1)
#         print(str(i)+AFLOWtherm['compound'][i]+'error')
#         mlcpaflowrelax[AFLOWtherm['compound'][i]+'_'+AFLOWtherm['ICSD'][i]]=0

In [54]:
# for comp in mlcpaflowrelax.keys():
#     if mlcpaflowrelax[comp]==0:
#         continue
#     else:
#         Xs=[]
#         Xp=[]
#         Xd=[]
#         Xf=[]
#         Xg=[]
#         for atom in range(len(mlcpaflowrelax[comp]['sites'])):
#             squaredcoeffs=[]
#             for con in mlcpaflowrelax[comp]['coeffs'][atom]:
#                 squaredcoeffs.append(con**2)
#             sumcoeffs=sum(squaredcoeffs)
#             if sumcoeffs==0:
#                 Xs.append(0)
#                 Xp.append(0)
#                 Xd.append(0)
#                 Xf.append(0)
#                 Xg.append(0)
#             else:
#                 Xs.append(squaredcoeffs[0]/sumcoeffs)
#                 Xp.append(sum(squaredcoeffs[1:4])/sumcoeffs)
#                 Xd.append(sum(squaredcoeffs[4:9])/sumcoeffs)
#                 Xf.append(sum(squaredcoeffs[9:16])/sumcoeffs)
#                 Xg.append(sum(squaredcoeffs[16:25])/sumcoeffs)
#         mlcpaflowrelax[comp]['Xs']=Xs
#         mlcpaflowrelax[comp]['Xp']=Xp
#         mlcpaflowrelax[comp]['Xd']=Xd
#         mlcpaflowrelax[comp]['Xf']=Xf
#         mlcpaflowrelax[comp]['Xg']=Xg

In [55]:
# np.save('mlcpdatafixedrelaxed.npy', mlcpaflowrelax) #4777 worked

In [149]:
mlcprelaxproblems=[]
for i in mlcpaflowrelax.keys():
    if mlcpaflowrelax[i]==0:
        mlcprelaxproblems.append(1)
    else:
        mlcprelaxproblems.append(0)
AFLOWtherm['relaxissues']=mlcprelaxproblems

mlcpproblems=[]
for i in mlcpaflow.keys():
    if mlcpaflow[i]==0:
        mlcpproblems.append(1)
    else:
        mlcpproblems.append(0)
AFLOWtherm['issues']=mlcpproblems

In [57]:
AFLOWtherm.to_pickle('AFLOWtherm.pkl')

In [3]:
AFLOWtherm = pd.read_pickle('AFLOWtherm.pkl')

In [4]:
mlcpaflow = np.load('mlcpdatafixed.npy',allow_pickle='TRUE').item()

In [5]:
mlcpaflowrelax = np.load('mlcpdatafixedrelaxed.npy',allow_pickle='TRUE').item()

In [6]:
AFLOWthermmlcp=AFLOWtherm.query('issues==0')
print(str(len(AFLOWthermmlcp))+' of '+str(len(AFLOWtherm))+' worked for original geometry')
AFLOWthermmlcprelax=AFLOWtherm.query('relaxissues==0')
print(str(len(AFLOWthermmlcprelax))+' of '+str(len(AFLOWtherm))+' worked for relaxed geometry')

4784 of 5578 worked for original geometry
4777 of 5578 worked for relaxed geometry


In [7]:
mlcpaflowresults={}
mlcpaflowrelaxresults={}
for i,j in zip(AFLOWthermmlcp['compound'],AFLOWthermmlcp['ICSD']):
    mlcpaflowresults[i+'_'+j]=mlcpaflow[i+'_'+j]
for i,j in zip(AFLOWthermmlcprelax['compound'],AFLOWthermmlcprelax['ICSD']):
    mlcpaflowrelaxresults[i+'_'+j]=mlcpaflowrelax[i+'_'+j]

In [8]:
redform=[]
CPmin=[]
CPminsite=[]
CPminatom=[]
CPminXs=[]
CPminavg=[]

CPmintherm=[]
CPoverall=[]

CPmultmin=[]
CPmultminsum=[]
CPmultminCP=[]
CPmultminsite=[]
CPmultminatom=[]
CPmultminXs=[]

CPminzscore=[]
CPminzscoreCP=[]
CPminzscoresite=[]
CPminzscoreatom=[]
CPminzscoreXs=[]

CPmultminavg=[]

CPmax=[]
CPmultmax=[]
CPmultminsumavg=[]
CPstandev=[]
CPmultstdev=[]
CPsumavg=[]
CPsum=[]
CPaverage=[]
CPmultaverage=[]
CPnegpct=[]
CPrange=[]
CPmultrange=[]

minXs=[]
maxXs=[]
rangeXs=[]
avgXs=[]
stdevXs=[]

minXp=[]
maxXp=[]
rangeXp=[]
avgXp=[]
stdevXp=[]

minXd=[]
maxXd=[]
rangeXd=[]
avgXd=[]
stdevXd=[]

minXf=[]
maxXf=[]
rangeXf=[]
avgXf=[]
stdevXf=[]

minXg=[]
maxXg=[]
rangeXg=[]
avgXg=[]
stdevXg=[]

memincp=[]
meminsite=[]
meminatom=[]
meminXeven=[]
meminXesum=[]
meminXesumavg=[]

CPmindiff=[]

for i,j in zip(AFLOWthermmlcp['compound'],AFLOWthermmlcp['ICSD']):
    struct=mlcpaflowresults[i+'_'+j]
    redform.append(struct['name'])
    minimumCP=min(struct['CP'])
    CPmin.append(minimumCP)
    CPmax.append(max(struct['CP']))
    CPrange.append(max(struct['CP'])-minimumCP)
    atomindex=struct['CP'].index(minimumCP)
    CPminsite.append(atomindex)
    CPminatom.append(struct['sites'][atomindex])
    CPoverall.append(struct['totalCP'])
    CPmean=np.mean(struct['CP'])
    CPstdev=np.std(struct['CP'])
    CPstandev.append(CPstdev)
    CPsum.append(sum(struct['CP']))
    CPsumavg.append(sum(struct['CP'])/len(struct['CP']))
    CPavg=sum([abs(c) for c in struct['CP']])/len(struct['CP'])
    CPaverage.append(CPavg)
    polarity=[1 if e<0 else 0 for e in struct['CP']]
    CPnegpct.append(sum(polarity)/len(polarity))
    CPminXs.append(struct['Xs'][atomindex])
    minmult=[a*b for a, b in zip(struct['Xs'], struct['CP'])]    
    CPmultstdev.append(np.std(minmult))
    CPmultaverage.append(sum([abs(c) for c in minmult])/len(minmult))
    CPmultrange.append(max(minmult)-min(minmult))
    minXs.append(min(struct['Xs']))
    maxXs.append(max(struct['Xs']))
    rangeXs.append(max(struct['Xs'])-min(struct['Xs']))
    avgXs.append(np.mean(struct['Xs']))
    stdevXs.append(np.std(struct['Xs']))
    minXp.append(min(struct['Xp']))
    maxXp.append(max(struct['Xp']))
    rangeXp.append(max(struct['Xp'])-min(struct['Xp']))
    avgXp.append(np.mean(struct['Xp']))
    stdevXp.append(np.std(struct['Xp']))
    minXd.append(min(struct['Xd']))
    maxXd.append(max(struct['Xd']))
    rangeXd.append(max(struct['Xd'])-min(struct['Xd']))
    avgXd.append(np.mean(struct['Xd']))
    stdevXd.append(np.std(struct['Xd']))
    minXf.append(min(struct['Xf']))
    maxXf.append(max(struct['Xf']))
    rangeXf.append(max(struct['Xf'])-min(struct['Xf']))
    avgXf.append(np.mean(struct['Xf']))
    stdevXf.append(np.std(struct['Xf']))
    minXg.append(min(struct['Xg']))
    maxXg.append(max(struct['Xg']))
    rangeXg.append(max(struct['Xg'])-min(struct['Xg']))
    avgXg.append(np.mean(struct['Xg']))
    stdevXg.append(np.std(struct['Xg']))
    multminCP=min(minmult)
    CPmultmin.append(multminCP)
    CPmultmax.append(max(minmult))
    CPmultminsum.append(sum(minmult))
    CPmultminsumavg.append(sum(minmult)/len(struct['CP']))
    minmultindex=minmult.index(multminCP)
    CPmultminsite.append(minmultindex)
    CPmultminatom.append(struct['sites'][minmultindex])
    CPmultminCP.append(struct['CP'][minmultindex])
    CPmultminXs.append(struct['Xs'][minmultindex])
    CPmultminavg.append(multminCP/CPavg)
    CPminavg.append(minimumCP/CPavg)
    if CPstdev==0:
        zscores=[0 for d in struct['CP']]
    else:
        zscores=[(d-CPmean)/CPstdev for d in struct['CP']]
    CPminzscore.append(min(zscores))
    Xeven=[x+y+z for x,y,z in zip(struct['Xs'],struct['Xd'],struct['Xg'])]
    eminmult=[a*b for a, b in zip(Xeven, struct['CP'])]
    memincp.append(min(eminmult))
    eminmultindex=eminmult.index(min(eminmult))
    meminatom.append(struct['sites'][eminmultindex])
    meminsite.append(minmultindex)
    meminXeven.append(Xeven[eminmultindex])
    meminXesum.append(sum(eminmult))
    meminXesumavg.append(sum(eminmult)/len(struct['CP']))
    CPmindiff.append(minimumCP-sum(struct['CP'])/len(struct['CP']))
    


In [9]:
rredform=[]
rCPmin=[]
rCPminsite=[]
rCPminatom=[]
rCPminXs=[]
rCPminavg=[]

rCPmintherm=[]
rCPoverall=[]

rCPmultmin=[]
rCPmultminsum=[]
rCPmultminCP=[]
rCPmultminsite=[]
rCPmultminatom=[]
rCPmultminXs=[]

rCPminzscore=[]
rCPminzscoreCP=[]
rCPminzscoresite=[]
rCPminzscoreatom=[]
rCPminzscoreXs=[]

rCPmultminavg=[]

rCPmax=[]
rCPmultmax=[]
rCPmultminsumavg=[]
rCPstandev=[]
rCPsumavg=[]
rCPsum=[]
rCPaverage=[]
rCPnegpct=[]
rCPrange=[]
rCPmultstdev=[]
rCPmultaverage=[]
rCPmultrange=[]

rminXs=[]
rmaxXs=[]
rrangeXs=[]
ravgXs=[]
rstdevXs=[]

rminXp=[]
rmaxXp=[]
rrangeXp=[]
ravgXp=[]
rstdevXp=[]

rminXd=[]
rmaxXd=[]
rrangeXd=[]
ravgXd=[]
rstdevXd=[]

rminXf=[]
rmaxXf=[]
rrangeXf=[]
ravgXf=[]
rstdevXf=[]

rminXg=[]
rmaxXg=[]
rrangeXg=[]
ravgXg=[]
rstdevXg=[]

rmemincp=[]
rmeminsite=[]
rmeminatom=[]
rmeminXeven=[]
rmeminXesum=[]
rmeminXesumavg=[]

rCPmindiff=[]

for i,j in zip(AFLOWthermmlcprelax['compound'],AFLOWthermmlcprelax['ICSD']):
    struct=mlcpaflowrelaxresults[i+'_'+j]
    rredform.append(struct['name'])
    rminimumCP=min(struct['CP'])
    rCPmin.append(rminimumCP)
    rCPmax.append(max(struct['CP']))
    rCPrange.append(max(struct['CP'])-rminimumCP)
    ratomindex=struct['CP'].index(rminimumCP)
    rCPminsite.append(ratomindex)
    rCPminatom.append(struct['sites'][ratomindex])
    rCPoverall.append(struct['totalCP'])
    rCPmean=np.mean(struct['CP'])
    rCPstdev=np.std(struct['CP'])
    rCPstandev.append(rCPstdev)
    rCPsum.append(sum(struct['CP']))
    rCPsumavg.append(sum(struct['CP'])/len(struct['CP']))
    rCPavg=sum([abs(c) for c in struct['CP']])/len(struct['CP'])
    rCPaverage.append(rCPavg)
    rCPminXs.append(struct['Xs'][ratomindex])
    rminmult=[a*b for a, b in zip(struct['Xs'], struct['CP'])]   
    rCPmultstdev.append(np.std(rminmult))
    rCPmultaverage.append(sum([abs(c) for c in rminmult])/len(rminmult))
    rCPmultrange.append(max(rminmult)-min(rminmult))
    rmultminCP=min(rminmult)
    rCPmultmin.append(rmultminCP)
    rCPmultminsum.append(sum(rminmult))
    rpolarity=[1 if e<0 else 0 for e in struct['CP']]
    rCPnegpct.append(sum(rpolarity)/len(rpolarity))
    rCPmultmax.append(max(rminmult))
    rminmultindex=rminmult.index(rmultminCP)
    rminXs.append(min(struct['Xs']))
    rmaxXs.append(max(struct['Xs']))
    rrangeXs.append(max(struct['Xs'])-min(struct['Xs']))
    ravgXs.append(np.mean(struct['Xs']))
    rstdevXs.append(np.std(struct['Xs']))
    rminXp.append(min(struct['Xp']))
    rmaxXp.append(max(struct['Xp']))
    rrangeXp.append(max(struct['Xp'])-min(struct['Xp']))
    ravgXp.append(np.mean(struct['Xp']))
    rstdevXp.append(np.std(struct['Xp']))
    rminXd.append(min(struct['Xd']))
    rmaxXd.append(max(struct['Xd']))
    rrangeXd.append(max(struct['Xd'])-min(struct['Xd']))
    ravgXd.append(np.mean(struct['Xd']))
    rstdevXd.append(np.std(struct['Xd']))
    rminXf.append(min(struct['Xf']))
    rmaxXf.append(max(struct['Xf']))
    rrangeXf.append(max(struct['Xf'])-min(struct['Xf']))
    ravgXf.append(np.mean(struct['Xf']))
    rstdevXf.append(np.std(struct['Xf']))
    rminXg.append(min(struct['Xg']))
    rmaxXg.append(max(struct['Xg']))
    rrangeXg.append(max(struct['Xg'])-min(struct['Xg']))
    ravgXg.append(np.mean(struct['Xg']))
    rstdevXg.append(np.std(struct['Xg']))
    rCPmultminsumavg.append(sum(rminmult)/len(struct['CP']))
    rCPmultminsite.append(rminmultindex)
    rCPmultminatom.append(struct['sites'][rminmultindex])
    rCPmultminCP.append(struct['CP'][rminmultindex])
    rCPmultminXs.append(struct['Xs'][rminmultindex])
    rCPmultminavg.append(rmultminCP/rCPavg)
    rCPminavg.append(rminimumCP/rCPavg)
    if rCPstdev==0:
        rzscores=[0 for d in struct['CP']]
    else:
        rzscores=[(d-rCPmean)/rCPstdev for d in struct['CP']]
    rCPminzscore.append(min(rzscores))
    rXeven=[x+y+z for x,y,z in zip(struct['Xs'],struct['Xd'],struct['Xg'])]
    reminmult=[a*b for a, b in zip(rXeven, struct['CP'])]
    rmemincp.append(min(reminmult))
    reminmultindex=reminmult.index(min(reminmult))
    rmeminatom.append(struct['sites'][reminmultindex])
    rmeminsite.append(rminmultindex)
    rmeminXeven.append(rXeven[reminmultindex])
    rmeminXesum.append(sum(reminmult))
    rmeminXesumavg.append(sum(reminmult)/len(struct['CP']))
    rCPmindiff.append(rminimumCP-sum(struct['CP'])/len(struct['CP']))
    

In [10]:
colorbracks=[]
kcolors=[]
kunder=[]

for i in AFLOWthermmlcp['kappa']:
    if i<=1:
        j='under1'
        k=5
        l='under1'
    elif i<=1.5:
        j='under1.5'
        k=4
        l='over1'
    elif i<=2:
        j='under2'
        k=3
        l='over1'
    elif i <=np.median(AFLOWthermmlcp['kappa']):
        j='undermedian'
        k=2
        l='over1'
    elif i<=6:
        j='under6'
        k=1
        l='over1'
    else:
        j='over6'
        k=0
        l='over1'
    colorbracks.append(j)
    kcolors.append(k)
    kunder.append(l)

rcolorbracks=[]
rkcolors=[]
rkunder=[]

for i in AFLOWthermmlcprelax['kappa']:
    if i<=1:
        j='under1'
        k=5
        l='under1'
    elif i<=1.5:
        j='under1.5'
        k=4
        l='over1'
    elif i<=2:
        j='under2'
        k=3
        l='over1'
    elif i <=np.median(AFLOWthermmlcprelax['kappa']):
        j='undermedian'
        k=2
        l='over1'
    elif i<=6:
        j='under6'
        k=1
        l='over1'
    else:
        j='over6'
        k=0
        l='over1'
    rcolorbracks.append(j)
    rkcolors.append(k)
    rkunder.append(l)

bias=[]
for i,j in zip(CPmin,CPmax):
    if i*j<0:
        bias.append(0)
    elif i<0:
        bias.append(max(i,j))
    elif i>0:
        bias.append(min(i,j))
    else:
        bias.append(0.01)

rbias=[]
for i,j in zip(rCPmin,rCPmax):
    if i*j<0:
        rbias.append(0)
    elif i<0 and j<0:
        rbias.append(max(i,j))
    elif i>0 and j>0:
        rbias.append(min(i,j))
    else:
        rbias.append(0.01)

In [11]:
AFLOWthermmlcp['rootkappa']=AFLOWthermmlcp['kappa']**0.5 #thermal conductivity square root
AFLOWthermmlcp['4rootkappa']=AFLOWthermmlcp['kappa']**0.25 #thermal conductivity tesseract root
AFLOWthermmlcp['ktiers']=kcolors #thermal conductivity discrete scale
AFLOWthermmlcp['kclass']=colorbracks #thermal conductivity categorical scale
AFLOWthermmlcp['kunder']=kunder #thermal conductivity under 1
AFLOWthermmlcp['reduced']=redform #reduced formula
AFLOWthermmlcp['totalcp']=CPoverall #l=0 magnitude of CP
AFLOWthermmlcp['mincp']=CPmin #lowest CP value
AFLOWthermmlcp['maxcp']=CPmax #max cp
AFLOWthermmlcp['rangecp']=CPrange #max cp - min cp
AFLOWthermmlcp['sumavgcp']=CPsumavg # avg cp
AFLOWthermmlcp['stdevcp']=CPstandev # stdev cp
AFLOWthermmlcp['sumcp']=CPsum # sum cp
AFLOWthermmlcp['Xs']=CPminXs #Xs at site
AFLOWthermmlcp['Xs2']=AFLOWthermmlcp['Xs']**2 #Xs at cite squared
AFLOWthermmlcp['avgcp']=CPaverage # avg mag cp
AFLOWthermmlcp['mincpavg']=CPminavg #mincp over avg magnitude of cp
AFLOWthermmlcp['mincpdiff']=CPmindiff #differenc between the lowest CP value and the average
AFLOWthermmlcp['mincpz']=CPminzscore #zscore of min cp
AFLOWthermmlcp['mmincp']=CPmultmin #min cp*Xs value
AFLOWthermmlcp['mmaxcp']=CPmultmax #max CP*Xs
AFLOWthermmlcp['mrangecp']=CPmultrange #max cp*Xs - min cp*Xs
AFLOWthermmlcp['msumavgcp']=CPmultminsumavg # cp*Xs average
AFLOWthermmlcp['mstdevcp']=CPmultstdev # stdev cp*Xs
AFLOWthermmlcp['msumcp']=CPmultminsum # sum cp*Xs
AFLOWthermmlcp['mmXs']=CPmultminXs #Xs of min cp*Xs value
AFLOWthermmlcp['mavgcp']=CPmultaverage # avg mag cp
AFLOWthermmlcp['mmcpavg']=CPmultminavg #mincp over avg magnitude of cp
AFLOWthermmlcp['mmcpcp']=CPmultminCP #cp of min cp*Xs value
AFLOWthermmlcp['memincp']=memincp #min (Xs+Xd)*CP value
AFLOWthermmlcp['mesumavgcp']=meminXesumavg #average of (Xs+Xd)*CP values
AFLOWthermmlcp['mesumcp']=meminXesum #sum of (Xs+Xd)*CP values
AFLOWthermmlcp['memXe']=meminXeven #min (Xs+Xd)*CP value (Xs+Xd) value
AFLOWthermmlcp['negpctcp']=CPnegpct #percent of net cps that are negative
AFLOWthermmlcp['bias']=bias #0 if there are both +/- net cp's, 0.01 if there are not only + or -, closest to zero if otherwise
AFLOWthermmlcp['minXs']=minXs #min Xs
AFLOWthermmlcp['maxXs']=maxXs #max Xs
AFLOWthermmlcp['rangeXs']=rangeXs #range Xs
AFLOWthermmlcp['avgXs']=avgXs #avg Xs
AFLOWthermmlcp['stdevXs']=stdevXs #stdev Xs
AFLOWthermmlcp['minXp']=minXp #min Xp
AFLOWthermmlcp['maxXp']=maxXp #max Xp
AFLOWthermmlcp['rangeXp']=rangeXp #range Xp
AFLOWthermmlcp['avgXp']=avgXp #avg Xp
AFLOWthermmlcp['stdevXp']=stdevXp #stdev Xp
AFLOWthermmlcp['minXd']=minXd #min Xd
AFLOWthermmlcp['maxXd']=maxXd #max Xd
AFLOWthermmlcp['rangeXd']=rangeXd #range Xd
AFLOWthermmlcp['avgXd']=avgXd #avg Xd
AFLOWthermmlcp['stdevXd']=stdevXd #stdev Xd
AFLOWthermmlcp['minXf']=minXf #min Xf
AFLOWthermmlcp['maxXf']=maxXf #max Xf
AFLOWthermmlcp['rangeXf']=rangeXf #range Xf
AFLOWthermmlcp['avgXf']=avgXf #avg Xf
AFLOWthermmlcp['stdevXf']=stdevXf #stdev Xf
AFLOWthermmlcp['minXg']=minXg #min Xg
AFLOWthermmlcp['maxXg']=maxXg #max Xg
AFLOWthermmlcp['rangeXg']=rangeXg #range Xg
AFLOWthermmlcp['avgXg']=avgXg #avg Xg
AFLOWthermmlcp['stdevXg']=stdevXg #stdev Xg
AFLOWthermmlcp['site']=CPminsite #site of the minimum CP value
AFLOWthermmlcp['atom']=CPminatom #atom at site
AFLOWthermmlcp['mmsite']=CPmultminsite #site of min cp*Xs value
AFLOWthermmlcp['mmatom']=CPmultminatom #atom of min cp*Xs value
AFLOWthermmlcp['memsite']=meminsite #min (Xs+Xd)*CP value site
AFLOWthermmlcp['mematom']=meminatom #min (Xs+Xd)*CP value atom

In [12]:
AFLOWthermmlcprelax['rootkappa']=AFLOWthermmlcprelax['kappa']**0.5 #thermal conductivity square root
AFLOWthermmlcprelax['4rootkappa']=AFLOWthermmlcprelax['kappa']**0.25 #thermal conductivity tesseract root
AFLOWthermmlcprelax['ktiers']=rkcolors #thermal conductivity discrete scale
AFLOWthermmlcprelax['kclass']=rcolorbracks #thermal conductivity categorical scale
AFLOWthermmlcprelax['kunder']=rkunder #thermal conductivity under 1
AFLOWthermmlcprelax['reduced']=rredform #reduced formula
AFLOWthermmlcprelax['totalcp']=rCPoverall #l=0 magnitude of CP
AFLOWthermmlcprelax['mincp']=rCPmin #lowest CP value
AFLOWthermmlcprelax['maxcp']=rCPmax #max cp
AFLOWthermmlcprelax['rangecp']=rCPrange #max cp - min cp
AFLOWthermmlcprelax['sumavgcp']=rCPsumavg # avg cp
AFLOWthermmlcprelax['stdevcp']=rCPstandev # stdev cp
AFLOWthermmlcprelax['sumcp']=rCPsum # sum cp
AFLOWthermmlcprelax['Xs']=rCPminXs #Xs at site
AFLOWthermmlcprelax['Xs2']=AFLOWthermmlcprelax['Xs']**2 #Xs at cite squared
AFLOWthermmlcprelax['avgcp']=rCPaverage # avg mag cp
AFLOWthermmlcprelax['mincpavg']=rCPminavg #mincp over avg magnitude of cp
AFLOWthermmlcprelax['mincpdiff']=rCPmindiff #differenc between the lowest CP value and the average
AFLOWthermmlcprelax['mincpz']=rCPminzscore #zscore of min cp
AFLOWthermmlcprelax['mmincp']=rCPmultmin #min cp*Xs value
AFLOWthermmlcprelax['mmaxcp']=rCPmultmax #max CP*Xs
AFLOWthermmlcprelax['mrangecp']=rCPmultrange #max cp*Xs - min cp*Xs
AFLOWthermmlcprelax['msumavgcp']=rCPmultminsumavg # cp*Xs average
AFLOWthermmlcprelax['mstdevcp']=rCPmultstdev # stdev cp*Xs
AFLOWthermmlcprelax['msumcp']=rCPmultminsum # sum cp*Xs
AFLOWthermmlcprelax['mmXs']=rCPmultminXs #Xs of min cp*Xs value
AFLOWthermmlcprelax['mavgcp']=rCPmultaverage # avg mag cp
AFLOWthermmlcprelax['mmcpavg']=rCPmultminavg #mincp over avg magnitude of cp
AFLOWthermmlcprelax['mmcpcp']=rCPmultminCP #cp of min cp*Xs value
AFLOWthermmlcprelax['memincp']=rmemincp #min (Xs+Xd)*CP value
AFLOWthermmlcprelax['mesumavgcp']=rmeminXesumavg #average of (Xs+Xd)*CP values
AFLOWthermmlcprelax['mesumcp']=rmeminXesum #sum of (Xs+Xd)*CP values
AFLOWthermmlcprelax['memXe']=rmeminXeven #min (Xs+Xd)*CP value (Xs+Xd) value
AFLOWthermmlcprelax['negpctcp']=rCPnegpct #percent of net cps that are negative
AFLOWthermmlcprelax['bias']=rbias #0 if there are both +/- net cp's, 0.01 if there are not only + or -, closest to zero if otherwise
AFLOWthermmlcprelax['minXs']=rminXs #min Xs
AFLOWthermmlcprelax['maxXs']=rmaxXs #max Xs
AFLOWthermmlcprelax['rangeXs']=rrangeXs #range Xs
AFLOWthermmlcprelax['avgXs']=ravgXs #avg Xs
AFLOWthermmlcprelax['stdevXs']=rstdevXs #stdev Xs
AFLOWthermmlcprelax['minXp']=rminXp #min Xp
AFLOWthermmlcprelax['maxXp']=rmaxXp #max Xp
AFLOWthermmlcprelax['rangeXp']=rrangeXp #range Xp
AFLOWthermmlcprelax['avgXp']=ravgXp #avg Xp
AFLOWthermmlcprelax['stdevXp']=rstdevXp #stdev Xp
AFLOWthermmlcprelax['minXd']=rminXd #min Xd
AFLOWthermmlcprelax['maxXd']=rmaxXd #max Xd
AFLOWthermmlcprelax['rangeXd']=rrangeXd #range Xd
AFLOWthermmlcprelax['avgXd']=ravgXd #avg Xd
AFLOWthermmlcprelax['stdevXd']=rstdevXd #stdev Xd
AFLOWthermmlcprelax['minXf']=rminXf #min Xf
AFLOWthermmlcprelax['maxXf']=rmaxXf #max Xf
AFLOWthermmlcprelax['rangeXf']=rrangeXf #range Xf
AFLOWthermmlcprelax['avgXf']=ravgXf #avg Xf
AFLOWthermmlcprelax['stdevXf']=rstdevXf #stdev Xf
AFLOWthermmlcprelax['minXg']=rminXg #min Xg
AFLOWthermmlcprelax['maxXg']=rmaxXg #max Xg
AFLOWthermmlcprelax['rangeXg']=rrangeXg #range Xg
AFLOWthermmlcprelax['avgXg']=ravgXg #avg Xg
AFLOWthermmlcprelax['stdevXg']=rstdevXg #stdev Xg
AFLOWthermmlcprelax['site']=rCPminsite #site of the minimum CP value
AFLOWthermmlcprelax['atom']=rCPminatom #atom at site
AFLOWthermmlcprelax['mmsite']=rCPmultminsite #site of min cp*Xs value
AFLOWthermmlcprelax['mmatom']=rCPmultminatom #atom of min cp*Xs value
AFLOWthermmlcprelax['memsite']=rmeminsite #min (Xs+Xd)*CP value site
AFLOWthermmlcprelax['mematom']=rmeminatom #min (Xs+Xd)*CP value atom

In [13]:
allformula=[]
allCP=[]
allisCPmin=[]
allsite=[]
allatom=[]
allXs=[]
alltherm=[]
# allXp=[]
# allXd=[]
# allXe=[]

rallformula=[]
rallCP=[]
rallisCPmin=[]
rallsite=[]
rallatom=[]
rallXs=[]
ralltherm=[]
# rallXp=[]
# rallXd=[]
# rallXe=[]


for k in AFLOWthermmlcprelax.index:
    i=AFLOWthermmlcprelax['compound'][k]
    j=AFLOWthermmlcprelax['ICSD'][k]
    rstruct=mlcpaflowrelaxresults[i+'_'+j]
    rminimumindex=rstruct['CP'].index(min(rstruct['CP']))
    for m in range(len(rstruct['sites'])):
        rallformula.append(rstruct['name'])
        rallCP.append(rstruct['CP'][m])
        if m==rminimumindex:
            rallisCPmin.append(1)
        else:
            rallisCPmin.append(0)
        rallsite.append(m)
        rallatom.append(rstruct['sites'][m])
        rallXs.append(rstruct['Xs'][m])
        ralltherm.append(AFLOWthermmlcprelax['kappa'][k])
        # rallXp.append(rstruct['Xp'][m])
        # rallXd.append(rstruct['Xd'][m])
        # rallXe.append(y+z for y,z in zip())
        

for k in AFLOWthermmlcp.index:
    i=AFLOWthermmlcp['compound'][k]
    j=AFLOWthermmlcp['ICSD'][k]
    struct=mlcpaflowresults[i+'_'+j]
    minimumindex=struct['CP'].index(min(struct['CP']))
    for n in range(len(struct['sites'])):
        allformula.append(struct['name'])
        allCP.append(struct['CP'][n])
        if n==minimumindex:
            allisCPmin.append(1)
        else:
            allisCPmin.append(0)
        allsite.append(n)
        allatom.append(struct['sites'][n])
        allXs.append(struct['Xs'][n])
        alltherm.append(AFLOWthermmlcp['kappa'][k])

In [14]:
allatoms = pd.DataFrame()
allatomsrelax = pd.DataFrame()

In [15]:
allatoms['formulas']=allformula
allatoms['cp']=allCP
allatoms['ismin']=allisCPmin
allatoms['site']=allsite
allatoms['atom']=allatom
allatoms['Xs']=allXs
allatoms['Xs2']=allatoms['Xs']**2
allatoms['mult']=allatoms['Xs']*allatoms['cp']
allatoms['kappa']=alltherm
allatoms['rootkappa']=allatoms['kappa']**0.5
allatoms['kunder']=[1 if x<1 else 0 for x in alltherm]


allatomsrelax['formulas']=rallformula
allatomsrelax['cp']=rallCP
allatomsrelax['ismin']=rallisCPmin
allatomsrelax['site']=rallsite
allatomsrelax['atom']=rallatom
allatomsrelax['Xs']=rallXs
allatomsrelax['Xs2']=allatomsrelax['Xs']**2
allatomsrelax['mult']=allatomsrelax['Xs']*allatomsrelax['cp']
allatomsrelax['kappa']=ralltherm
allatomsrelax['rootkappa']=allatomsrelax['kappa']**0.5
allatomsrelax['kunder']=[1 if x<1 else 0 for x in ralltherm]

In [16]:
AFLOWthermmlcp2=AFLOWthermmlcp.drop(45)

In [17]:
forbiddens15=['H','He','F','Ne','Cl','Ar','Br','Kr','I','Xe']
forbiddens1=['Li','Na','K','Rb','Cs'] #can keep
forbiddens16=['O','S','Se','Te'] #can keep
forbiddens=forbiddens15

In [18]:
wrong_counter=0
mlcpelements=[]
notmlcpelements=[]
for compound in AFLOWthermmlcp2.index:
    compositional=Composition(AFLOWthermmlcp2.loc[compound,'compound']).element_composition.as_dict().keys()
    if any(i in forbiddens for i in compositional)==True:
        notmlcpelements.append(compound)
        wrong_counter+=1
    else:
        mlcpelements.append(compound)

rwrong_counter=0
rmlcpelements=[]
rnotmlcpelements=[]
for compound in AFLOWthermmlcprelax.index:
    compositional=Composition(AFLOWthermmlcprelax.loc[compound,'compound']).element_composition.as_dict().keys()
    if any(i in forbiddens for i in compositional)==True:
        rnotmlcpelements.append(compound)
        rwrong_counter+=1
    else:
        rmlcpelements.append(compound)

ATMNE=AFLOWthermmlcp2.loc[notmlcpelements]
ATME=AFLOWthermmlcp2.loc[mlcpelements]
ATMNER=AFLOWthermmlcprelax.loc[rnotmlcpelements]
ATMER=AFLOWthermmlcprelax.loc[rmlcpelements]

In [19]:
AFLOWthermmlcp2.to_pickle('ATM.pkl')
AFLOWthermmlcprelax.to_pickle('ATMR.pkl')
ATMNE.to_pickle('ATMNE.pkl')
ATME.to_pickle('ATME.pkl')
ATMNER.to_pickle('ATMNER.pkl')
ATMER.to_pickle('ATMER.pkl')
allatoms.to_pickle('allatoms.pkl')
allatomsrelax.to_pickle('allatomsrelax.pkl')

In [20]:
ATM = pd.read_pickle('ATM.pkl')
ATMR = pd.read_pickle('ATMR.pkl')
ATMNE = pd.read_pickle('ATMNE.pkl')
ATME = pd.read_pickle('ATME.pkl')
ATMNER = pd.read_pickle('ATMNER.pkl')
ATMER = pd.read_pickle('ATMER.pkl')
allatoms = pd.read_pickle('allatoms.pkl')
allatomsrelax = pd.read_pickle('allatomsrelax.pkl')

In [21]:
ATMDATA={}
ATMDATA['ATM']=ATM
ATMDATA['ATMR']=ATMR
ATMDATA['ATME']=ATME
ATMDATA['ATMER']=ATMER
ATMDATA['ATMNE']=ATMNE
ATMDATA['ATMNER']=ATMNER

In [22]:
for i in ['ATM','ATME','ATMNE','ATMR','ATMER','ATMNER']:
    ATMDATA[i+'M']=ATMDATA[i].query('Etype==0')
    ATMDATA[i+'S']=ATMDATA[i].query('Etype==1')
    ATMDATA[i+'I']=ATMDATA[i].query('Etype==2')
for i in ['ATM','ATME','ATMNE','ATMR','ATMER','ATMNER']:
    ATMDATA[i+'2']=ATMDATA[i].query('nspecies<=2')
    ATMDATA[i+'2']=ATMDATA[i+'M'].query('nspecies<=2')
    ATMDATA[i+'2']=ATMDATA[i+'S'].query('nspecies<=2')
    ATMDATA[i+'2']=ATMDATA[i+'I'].query('nspecies<=2')

In [23]:
ATMDATA.keys()

dict_keys(['ATM', 'ATMR', 'ATME', 'ATMER', 'ATMNE', 'ATMNER', 'ATMM', 'ATMS', 'ATMI', 'ATMEM', 'ATMES', 'ATMEI', 'ATMNEM', 'ATMNES', 'ATMNEI', 'ATMRM', 'ATMRS', 'ATMRI', 'ATMERM', 'ATMERS', 'ATMERI', 'ATMNERM', 'ATMNERS', 'ATMNERI', 'ATM2', 'ATME2', 'ATMNE2', 'ATMR2', 'ATMER2', 'ATMNER2'])